# Empreinte CO2 d'une recette

## Import des bibliothèques

In [41]:
import requests
import pandas as pd
import json

ENDPOINT = "http://162.19.108.77:8000"

## Creation de la requête pour accéder à une liste de recette

In [30]:
url = f'{ENDPOINT}/recipe_urls'
body = {"name": "pates carbonara"}
r = requests.post(url, json=body)

In [31]:
r.status_code

200

In [32]:
r.content

b'["https://www.marmiton.org/recettes/recette_pates-a-la-carbonara_80453.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara_21229.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara-la-vraie-recette-italienne_17213.aspx","https://www.marmiton.org/recettes/recette_pates-a-la-carbonara-et-aux-legumes-du-soleil_58989.aspx","https://www.marmiton.org/recettes/recette_pates-a-la-carbonara-facon-indienne_63815.aspx","https://www.marmiton.org/recettes/recette_pates-papillon-courgette_535270.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara-all-italiana_312820.aspx","https://www.marmiton.org/recettes/recette_gratin-de-pates-facon-carbonara_168112.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara-aux-poireaux_334939.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara-feuilletees_344120.aspx","https://www.marmiton.org/recettes/recette_pates-carbonara-a-la-flo-sans-oeuf_58895.aspx","https://www.marmiton.org/recettes/recette_pate

In [33]:
data = r.json()
data

['https://www.marmiton.org/recettes/recette_pates-a-la-carbonara_80453.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara_21229.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara-la-vraie-recette-italienne_17213.aspx',
 'https://www.marmiton.org/recettes/recette_pates-a-la-carbonara-et-aux-legumes-du-soleil_58989.aspx',
 'https://www.marmiton.org/recettes/recette_pates-a-la-carbonara-facon-indienne_63815.aspx',
 'https://www.marmiton.org/recettes/recette_pates-papillon-courgette_535270.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara-all-italiana_312820.aspx',
 'https://www.marmiton.org/recettes/recette_gratin-de-pates-facon-carbonara_168112.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara-aux-poireaux_334939.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara-feuilletees_344120.aspx',
 'https://www.marmiton.org/recettes/recette_pates-carbonara-a-la-flo-sans-oeuf_58895.aspx',
 'https://www.marmiton.org/r

In [51]:
data[2]

'https://www.marmiton.org/recettes/recette_pates-carbonara-la-vraie-recette-italienne_17213.aspx'

## Requete pour acceder à la liste d'ingrédients

In [57]:
url = f'{ENDPOINT}/ingredients_marmiton/'
body2 = {"url": data[2]}
r2= requests.post(url, json=body2)

In [58]:
r2.status_code

200

In [59]:

r2.content

b'["250\xc2\xa0g de lardons "," poivre "," sel ","500\xc2\xa0g de spaghetti ","6 oeufs "," Parmesan "]'

In [60]:
data2 = r2.json()
data2

['250\xa0g de lardons ',
 ' poivre ',
 ' sel ',
 '500\xa0g de spaghetti ',
 '6 oeufs ',
 ' Parmesan ']

## Obtention du code AGB

J'ouvre mon fichier credentials.json

In [42]:
with open("credentials.json") as f:
    credentials = json.load(f)
    token = credentials["token"]

Je vérifie mon quota de jetons

In [61]:
url = ENDPOINT +"/quota/"
headers = {"Authorization" : f"Token {token}"}

r = requests.get(url, headers = headers)
r.status_code
r.json()

'Quota used: 1/3'

J'effectue ma requete pour obtenir les code AGB de ma recette

In [62]:
url = f'{ENDPOINT}/ingredients_recipe/'
code_AGB = {"ingredients": data2}
# ou --> code_AGB = {"name": "pates carbonara", "url":data}
headers = {"Authorization" : f"Token {token}"}

r3 = requests.post(url, json=code_AGB, headers = headers)
r3.status_code

200

In [63]:
r3.json()

'[{"ingredient":"lardons","Code\\nAGB":"28720","mass":250},{"ingredient":"poivre","Code\\nAGB":"11015","mass":0},{"ingredient":"sel","Code\\nAGB":"11017","mass":0},{"ingredient":"spaghetti","Code\\nAGB":"9610","mass":500},{"ingredient":"oeufs","Code\\nAGB":"22000","mass":0},{"ingredient":"Parmesan","Code\\nAGB":"12120","mass":0}]'

J'importe ma liste d'ingredients dans un dataframe et je le suvegarde dans un fichier csv

In [64]:
ingredients_recipe = pd.read_json(r3.json(), orient="records")
ingredients_recipe.to_csv("datas/pates_carbonara.csv", sep = ";", index = False)
ingredients_recipe

,ingredient,Code\nAGB,mass
0,lardons,28720,250
1,poivre,11015,0
2,sel,11017,0
3,spaghetti,9610,500
4,oeufs,22000,0
5,Parmesan,12120,0


## Fusion de dataframe pour obtenir les quantité de CO2 par code AGB des ingrédients

In [78]:
df_synthese_full = pd.read_csv("datas/synthese_ingredient.csv")

df_synthese_full.dtypes


Code\nAGB                                                                      object
Code\nCIQUAL                                                                    int64
Groupe d'aliment                                                               object
Sous-groupe d'aliment                                                          object
Nom du Produit en Français                                                     object
LCI Name                                                                       object
code saison (0 : hors saison ; 1 : de saison ; 2 : mix de consommation FR)      int64
code avion (1 : par avion)                                                      int64
Livraison                                                                      object
Matériau d'emballage                                                           object
Préparation                                                                    object
DQR - Note de qualité de la donnée (1 excellente ; 5 t

In [79]:
ingredients_recipe.dtypes

ingredient    object
Code\nAGB      int64
mass           int64
dtype: object

Je veux changer le type des colonnes Code\nAGB dans les 2 dataframes au formats string

In [81]:


df_synthese_full['Code\nAGB'] = df_synthese_full['Code\nAGB'].astype(str)
ingredients_recipe['Code\nAGB'] = ingredients_recipe['Code\nAGB'].astype(str)

In [85]:
fusion = pd.merge(df_synthese_full, ingredients_recipe, how = 'inner')
fusion

,Code\nAGB,Code\nCIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,code saison (0 : hors saison ; 1 : de saison ; 2 : mix de consommation FR),code avion (1 : par avion),Livraison,Matériau d'emballage,...,kg P eq/kg de produit,kg N eq/kg de produit,mol N eq/kg de produit,CTUe/kg de produit,Pt/kg de produit,m3 depriv./kg de produit,MJ/kg de produit,kg Sb eq/kg de produit,ingredient,mass
0,11015,11015,aides culinaires et ingrédients divers,épices,"Poivre noir, poudre","Black pepper, powder",2,0,Ambiant (long),Verre,...,0.001586,0.069950,0.579813,164.114710,698.971520,19.200214,87.472257,0.000129,poivre,0
1,11017,11017,aides culinaires et ingrédients divers,sels,"Sel blanc alimentaire, non iodé, non fluoré (m...","Salt, white, for human consumption (sea, igneo...",2,0,Ambiant (long),PVC,...,0.000213,0.000715,0.006380,7.027442,3.902153,0.414198,10.826810,0.000018,sel,0
2,12120,12120,lait et produits laitiers,fromages,Parmesan,"Parmesan cheese, from cow's milk",2,0,Glacé,LDPE,...,0.000848,0.020979,0.268319,63.207046,333.434270,2.145010,33.308480,0.000016,Parmesan,0
3,9610,9610,produits céréaliers,"pâtes, riz et céréales","Semoule de blé dur, crue","Durum wheat semolina, raw",2,0,Ambiant (long),LDPE,...,0.000645,0.016089,0.071268,11.824564,199.190700,1.349348,21.470370,0.000010,spaghetti,500
4,28720,28720,"viandes, œufs, poissons",charcuteries,"Lardon fumé, cru","Smoked lardoons, raw",2,0,Glacé,PS,...,0.000942,0.029005,0.526850,99.183280,354.753710,4.284565,68.479801,0.000021,lardons,250
5,22000,22000,"viandes, œufs, poissons",œufs,"Oeuf, cru","Egg, raw",2,0,Ambiant (moyen),Carton,...,0.000826,0.018525,0.363972,80.170819,235.755740,4.310059,20.820181,0.000012,oeufs,0


In [ ]:
fusion.columns

In [87]:
fusion["Impact Carbone de l'ingrédient"] = fusion['kg CO2 eq/kg de produit'] * fusion['mass'] / 1000
fusion

,Code\nAGB,Code\nCIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,code saison (0 : hors saison ; 1 : de saison ; 2 : mix de consommation FR),code avion (1 : par avion),Livraison,Matériau d'emballage,...,kg N eq/kg de produit,mol N eq/kg de produit,CTUe/kg de produit,Pt/kg de produit,m3 depriv./kg de produit,MJ/kg de produit,kg Sb eq/kg de produit,ingredient,mass,Impact Carbone de l'ingrédient
0,11015,11015,aides culinaires et ingrédients divers,épices,"Poivre noir, poudre","Black pepper, powder",2,0,Ambiant (long),Verre,...,0.069950,0.579813,164.114710,698.971520,19.200214,87.472257,0.000129,poivre,0,0.000000
1,11017,11017,aides culinaires et ingrédients divers,sels,"Sel blanc alimentaire, non iodé, non fluoré (m...","Salt, white, for human consumption (sea, igneo...",2,0,Ambiant (long),PVC,...,0.000715,0.006380,7.027442,3.902153,0.414198,10.826810,0.000018,sel,0,0.000000
2,12120,12120,lait et produits laitiers,fromages,Parmesan,"Parmesan cheese, from cow's milk",2,0,Glacé,LDPE,...,0.020979,0.268319,63.207046,333.434270,2.145010,33.308480,0.000016,Parmesan,0,0.000000
3,9610,9610,produits céréaliers,"pâtes, riz et céréales","Semoule de blé dur, crue","Durum wheat semolina, raw",2,0,Ambiant (long),LDPE,...,0.016089,0.071268,11.824564,199.190700,1.349348,21.470370,0.000010,spaghetti,500,0.833739
4,28720,28720,"viandes, œufs, poissons",charcuteries,"Lardon fumé, cru","Smoked lardoons, raw",2,0,Glacé,PS,...,0.029005,0.526850,99.183280,354.753710,4.284565,68.479801,0.000021,lardons,250,1.597019
5,22000,22000,"viandes, œufs, poissons",œufs,"Oeuf, cru","Egg, raw",2,0,Ambiant (moyen),Carton,...,0.018525,0.363972,80.170819,235.755740,4.310059,20.820181,0.000012,oeufs,0,0.000000


In [94]:
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

couleur = px.colors.qualitative.Dark24[:fusion["Sous-groupe d'aliment"].nunique()]


graph = go.Bar(x=fusion["ingredient"], y=fusion["Impact Carbone de l'ingrédient"], marker=dict(color=couleur)) 

fig = go.Figure(data=[graph])

# Set y-axis label for the middle subplot
fig.update_yaxes(title_text="kg CO2 eq/kg de produit")

iplot(fig)

In [95]:
data = go.Pie(labels=fusion["ingredient"], values=fusion["Impact Carbone de l'ingrédient"], marker=dict(colors=couleur))
layout = go.Layout(title="Proportions de l'impact carbone de chaque ingrédient")

fig = go.Figure(data=[data], layout=layout)
iplot(fig)
